## 5.1 학습 모델 저장하고 재사용하기

#### .csv 파일로 데이터 분리하기. 1열은 털, 2열은 날개, 3 ~ 마지막 열은 One-hot 인코딩으로 개체의 종류를 나타내는 데이터이다.

In [1]:
import tensorflow as tf
import numpy as np

#### 같은 디렉토리에 data.csv 파일을 둔다.

In [2]:
data = np.loadtxt('./data.csv', delimiter = ',', dtype = 'float32')

x_data = data[:, 0:2]
y_data = data[:, 2:]

In [3]:
x_data

array([[0., 0.],
       [1., 0.],
       [1., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.]], dtype=float32)

In [4]:
y_data

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [5]:
### Variable for counting training number
global_step = tf.Variable(0, trainable = False, name = 'global_step')

In [6]:
### Placeholder & Weight Setting
### For this time, there is no bias
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

### Neural Net Setting
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

#### Bias는 없고, 은닉층은 2층인 DNN. 1층은 10개, 2층은 20개

In [7]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits = model))

optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
train_op = optimizer.minimize(cost, global_step = global_step)

#### train_op에서 앞서 정의한 global_step을 global_step 파라미터에 전달하였다. 이렇게 하면 최적화 함수가 학습용 변수들을 최적화할 때마다, global_step 값을 1씩 증가한다.

In [8]:
sess = tf.Session()
### Function that brings pre-defined variables 
saver = tf.train.Saver(tf.global_variables())

#### global_variables()를 통해 앞서 정의한 변수들을 모두 가져와서, 이후 이 변수들을 파일에 저장하거나 이전에 학습한 결과를 불러와 담는 변수들로 사용한다.

In [9]:
ckpt = tf.train.get_checkpoint_state('./model')
### Check whether the trained model exists in ./model directory
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    ### Load trained model
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    ### Initialize variables
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model\dnn.ckpt-6


#### 학습된 모델을 저장한 파일을 체크포인트 파일이라 한다. 이번에는 step 값이 아니라 global_step값을 이용해 학습을 몇 번째 진행하고 있는지 확인한다.

In [10]:
for step in range(2):
    sess.run(train_op, feed_dict = {X: x_data, Y: y_data})
    
    print("Step: ", sess.run(global_step), "Cost: ", sess.run(cost, feed_dict = {X: x_data, Y: y_data}))

Step:  1 Cost:  0.639864
Step:  2 Cost:  0.6262947


In [11]:
### Save trained variables in checkpoint file
saver.save(sess, './model/dnn.ckpt', global_step = global_step)

'./model/dnn.ckpt-2'

In [12]:
### Printing Prediction & Real
prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print ("예측값: ", sess.run(prediction, feed_dict = {X: x_data}))
print ("실제값: ", sess.run(target, feed_dict = {Y: y_data}))

### Compute Accuracy
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ("정확도: ", sess.run(accuracy * 100, feed_dict = {X: x_data, Y: y_data}))

예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도:  100.0


#### 이번 코드를 통해 텐서플로우로 모델 구성은 물론, 학습 후 이것을 저장 및 로드하여 재사용할 수 있음을 확인할 수 있었다.

#### 로드 후 재사용이 가능하다는 의미는, 학습과 예측을 분리하여 실행할 수 있음을 의미한다.

## 5.2 텐서보드 사용하기

#### 텐서보드는 학습 과정을 추적하기 쉽게, 손실값이나 정확도 등을 다양한 방식으로 시각화하여 보여준다.

In [13]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter = ',', dtype = 'float32')

x_data = data[:, 0:2]
y_data = data[:, 2:]

### Variable for counting training number
global_step = tf.Variable(0, trainable = False, name = 'global_step')

### Placeholder & Weight Setting
### For this time, there is no bias
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

#### 텐서보드를 사용하기 위한 코드를 추가한다.

In [14]:
with tf.name_scope('layer1'):
    ### Neural Net Setting
    ### You can check implicitly in tensorboard
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name = 'W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name = 'W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name = 'W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits = model))

    optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
    train_op = optimizer.minimize(cost, global_step = global_step)
    
    tf.summary.scalar('cost', cost)

##### 값이 하나인 텐서를 수집할 때 scalar 메소드를 사용한다.

In [15]:
sess = tf.Session()
### Function that brings pre-defined variables 
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model2')
### Check whether the trained model exists in ./model directory
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    ### Load trained model
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    ### Initialize variables
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model2\dnn.ckpt-100


In [16]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

#### tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음, tf.summary.FileWriter 함수를 이용해 그래프와 텐서들의 값을 저장할 디렉토리를 설정한다.

In [17]:
for step in range(100):
    sess.run(train_op, feed_dict = {X: x_data, Y: y_data})
    
    print("Step: ", sess.run(global_step), "Cost: ", sess.run(cost, feed_dict = {X: x_data, Y: y_data}))
    
    summary = sess.run(merged, feed_dict = {X: x_data, Y: y_data})
    writer.add_summary(summary, global_step = sess.run(global_step))

Step:  1 Cost:  1.2285805
Step:  2 Cost:  1.1200504
Step:  3 Cost:  1.0265937
Step:  4 Cost:  0.9490139
Step:  5 Cost:  0.88534784
Step:  6 Cost:  0.8362914
Step:  7 Cost:  0.7956496
Step:  8 Cost:  0.76097244
Step:  9 Cost:  0.73139685
Step:  10 Cost:  0.7070139
Step:  11 Cost:  0.6864138
Step:  12 Cost:  0.6682369
Step:  13 Cost:  0.6522586
Step:  14 Cost:  0.63922024
Step:  15 Cost:  0.62790966
Step:  16 Cost:  0.6181378
Step:  17 Cost:  0.6097264
Step:  18 Cost:  0.6026749
Step:  19 Cost:  0.5965258
Step:  20 Cost:  0.59115857
Step:  21 Cost:  0.58634025
Step:  22 Cost:  0.58201265
Step:  23 Cost:  0.5782985
Step:  24 Cost:  0.57502705
Step:  25 Cost:  0.5721347
Step:  26 Cost:  0.5695888
Step:  27 Cost:  0.56739146
Step:  28 Cost:  0.56547564
Step:  29 Cost:  0.56380373
Step:  30 Cost:  0.5623707
Step:  31 Cost:  0.56111544
Step:  32 Cost:  0.56003016
Step:  33 Cost:  0.55910677
Step:  34 Cost:  0.5582829
Step:  35 Cost:  0.5575461
Step:  36 Cost:  0.5568865
Step:  37 Cost:  0.556

#### merged에 모아둔 텐서의 값들을 계산하여 summary에 수집한 뒤, add_summary 함수로 변수 writer에 해당 값들을 지정한 디렉토리에 저장한다.

In [18]:
### Save trained variables in checkpoint file
saver.save(sess, './model2/dnn.ckpt', global_step = global_step)

### Printing Prediction & Real
prediction = tf.argmax(model, axis = 1)
target = tf.argmax(Y, axis = 1)
print ("예측값: ", sess.run(prediction, feed_dict = {X: x_data}))
print ("실제값: ", sess.run(target, feed_dict = {Y: y_data}))

### Compute Accuracy
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print ("정확도: ", sess.run(accuracy * 100, feed_dict = {X: x_data, Y: y_data}))

예측값:  [0 1 2 0 0 2]
실제값:  [0 1 2 0 0 2]
정확도:  100.0
